In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk import SnowballStemmer
import re, string
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas.core.frame import DataFrame

df = pd.read_csv('dataset/train.csv')
df1 = pd.read_csv(('dataset/test.csv'))
df.comment_text = df.comment_text.fillna('')
df1.comment_text = df1.comment_text.fillna('')



In [4]:
stops = set(stopwords.words("english"))
stops2 = set(stopwords.words("french"))
stops3 = set(stopwords.words("spanish"))
train = df.comment_text
test = df1.comment_text
special_character_removal = re.compile(r'[^a-z\d ]', re.IGNORECASE)
replace_numbers = re.compile(r'\d+',re.IGNORECASE)

In [5]:
def remove(text):
    text = text.lower().split()
    text = [w for w in text if not w in stops]
    text = [w for w in text if not w in stops2]
    text = [w for w in text if not w in stops3]
    text = " ".join(text)
    text = special_character_removal.sub('', text)
    text = replace_numbers.sub('n', text)

    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

In [6]:
trainafter = []
for text in train:
    a = remove(text)
    trainafter.append(a)
testafter = []
for text in test:
    b = remove(text)
    testafter.append(b)

In [10]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()
cv = TfidfVectorizer(ngram_range=(2,2),tokenizer=tokenize,min_df=3,max_df=0.9,strip_accents='unicode',
                     use_idf=1,smooth_idf=1,sublinear_tf=1)
cv.fit(trainafter)
train_data = cv.transform(trainafter)
test_data = cv.transform(testafter)

In [63]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

submission = pd.DataFrame.from_dict({'id': df1['id']})
score = []
accuracy = []
precision = []
av_precision = []
recall = []
f1micro = []
f1macro = []
fmeasure = []
for label in  class_names:
    train_label = df[label].astype('int')
    
    mnb = MultinomialNB(alpha = 0.01)
    cv_score = np.mean(cross_val_score(mnb, train_data, train_label, cv=3,scoring='roc_auc'))
    #score.append(cv_score)
    accuracy_e = np.mean(cross_val_score(mnb, train_data, train_label, cv=3,scoring='accuracy'))
    accuracy.append(accuracy_e)
    pre = np.mean(cross_val_score(mnb, train_data, train_label, cv=3,scoring='precision'))
    precision.append(pre)
    av = np.mean(cross_val_score(mnb, train_data, train_label, cv=3,scoring='average_precision'))
    av_precision.append(av)
    recall_e = np.mean(cross_val_score(mnb, train_data, train_label, cv=3,scoring='recall'))
    recall.append(recall_e)
    f1 = np.mean(cross_val_score(mnb, train_data, train_label, cv=3,scoring='f1_micro'))
    f1micro.append(f1)
    f2 = np.mean(cross_val_score(mnb, train_data, train_label, cv=3,scoring='f1_macro'))
    f1macro.append(f2)
    fm = np.mean(cross_val_score(mnb, train_data, train_label, cv=3,scoring='f1'))
    fmeasure.append(fm)
    mnb.fit(train_data,train_label)
    #print(label,cv_score)
    submission[label] = mnb.predict_proba(test_data)[:,1]
    score.append(cv_score)

submission.to_csv('submission.csv',index=False)

In [66]:
d = {
    'Label': class_names,
    'F-measure': fmeasure,
    'F-measure(micro)':f1micro,
    'F-measure(macro)':f1macro,
    'ROC_AUC': score,
    'Precision': precision,
    'Recall': recall
}
e = {
    'Label': class_names,
    'Accuracy of each class': accuracy
}
result1 = DataFrame(e)
col = ['Label', 'Accuracy of each class']
result1 = result1.ix[:, col]
result2 = DataFrame(d)
cols = ['Label','Precision', 'Recall','F-measure', 'F-measure(macro)', 'F-measure(micro)', 'ROC_AUC']
result2 = result2.ix[:, cols]
result2

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,Label,Precision,Recall,F-measure,F-measure(macro),F-measure(micro),ROC_AUC
0,toxic,0.781533,0.599189,0.678319,0.824282,0.945529,0.935995
1,severe_toxic,0.405038,0.410029,0.407492,0.700737,0.988087,0.962344
2,obscene,0.728810,0.603621,0.660276,0.821498,0.967112,0.942637
3,threat,0.382914,0.234237,0.289065,0.643670,0.996560,0.933799
4,insult,0.674745,0.568998,0.617278,0.799518,0.965175,0.945129
5,identity_hate,0.344369,0.229876,0.275125,0.634881,0.989353,0.917378


In [45]:
want = False
if want == True:
    acc = pd.read_csv('submission.csv')
    #proba = acc.toxic
    def zero(acc):
        zero = []
        c = 0
        for i in acc:
            if i>0.9:
                i=1
            else:
                i=0
            zero.append(i)
        return zero
    
    fsubmission = pd.DataFrame.from_dict({'id': df1['id']})
    for label in class_names:
        fsubmission[label] = zero(acc[label])
        fsubmission.to_csv('fsubmission.csv',index=False)